In [1]:
import pandas as pd

In [2]:
paises = pd.read_csv('paises.csv', sep=';') 
ciudades = pd.read_csv('ciudades.csv', sep=';')
departamentos = pd.read_csv('departamento.csv', sep=';')
motivo = pd.read_csv('motivo de viaje.csv', sep=';')
ocupacion = pd.read_csv('ocupaciones.csv', sep=';')
transporte = pd.read_csv('medio transporte.csv', sep=';')

In [3]:
columns = ['VIAC_EMIGRA_INMIGRA', 'VIAN_NACIONALIDAD',
       'VIAC_SEXO', 'VIAN_MOTIVO_VIAJE',
       'VIAF_VIAJE',  'VIAN_PAIS_DESTINO','VIAN_DEPTO_DESTINO','VIAN_CIUDAD_DESTINO','VIAC_OCUPACION', 
        'VIAC_ESTADO', 'VIAC_MEDIO_TRANSPORTE',
       'RANGO_EDAD', 'REGION_DESTINO',
       'REGION_NACIONALIDAD']

In [4]:
e_combined_parts = []

files = [
    'Flujos_Migratorios_Anomimizado_2018.csv',
    'Flujos_Migratorios_Anomimizado_2019.csv',
    'Flujos_Migratorios_Anomimizado_2020.csv',
    'Flujos_Migratorios_Anomimizado_2021.csv'
]

for file in files:
    # Leemos el archivo en fragmentos
    for chunk in pd.read_csv(file, sep=';', usecols=columns, chunksize=100000):  # Ajusta el chunksize si es necesario
        # Filtramos los datos y los añadimos a las listas de cada categoría
        e_combined_parts.append(chunk[chunk['VIAC_EMIGRA_INMIGRA'] == 'E'])
        
# Concatenamos los datos        
e_combined = pd.concat(e_combined_parts, ignore_index=True) 

del e_combined_parts # Liberamos memoria      

C:\Users\Andrés Dardex\AppData\Local\Temp\ipykernel_15336\1642215523.py:12: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file, sep=';', usecols=columns, chunksize=100000):  # Ajusta el chunksize si es necesario
C:\Users\Andrés Dardex\AppData\Local\Temp\ipykernel_15336\1642215523.py:12: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file, sep=';', usecols=columns, chunksize=100000):  # Ajusta el chunksize si es necesario
C:\Users\Andrés Dardex\AppData\Local\Temp\ipykernel_15336\1642215523.py:12: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file, sep=';', usecols=columns, chunksize=100000):  # Ajusta el chunksize si es necesario
C:\Users\Andrés Dardex\AppData\Local\Temp\ipykernel_15336\1642215523.py:12: DtypeWarning: Columns (13) have mixed type

In [5]:
e_combined.drop_duplicates(inplace=True)

In [6]:
ciudades_dict = {}

for index, row in ciudades.iterrows():
    pais = row['CIUN_PAIS']
    departamento = row['CIUN_DEPARTAMENTO']
    ciudad_codigo = row['CIUN_CODIGO']
    ciudad_nombre = row['CIUC_NOMBRE']
    
    # Si el país no está en el diccionario, agregarlo
    if pais not in ciudades_dict:
        ciudades_dict[pais] = {}
    
    # Si el departamento no está en el país, agregarlo
    if departamento not in ciudades_dict[pais]:
        ciudades_dict[pais][departamento] = {}
    
    # Asignar el nombre de la ciudad al código de ciudad
    ciudades_dict[pais][departamento][ciudad_codigo] = ciudad_nombre

In [7]:
# Función para reemplazar los códigos de ciudad por los nombres
def reemplazar_ciudad(row):
    pais = row['VIAN_PAIS_DESTINO']
    departamento = row['VIAN_DEPTO_DESTINO']
    ciudad_codigo = row['VIAN_CIUDAD_DESTINO']
    
    # Verificar si el país y departamento están en el diccionario
    if pais in ciudades_dict and departamento in ciudades_dict[pais]:
        # Obtener el nombre de la ciudad
        return ciudades_dict[pais][departamento].get(ciudad_codigo, 'Ciudad desconocida')
    return 'Ciudad desconocida'

# Reemplazar los códigos de ciudad por nombres directamente en el DataFrame
e_combined['VIAN_CIUDAD_DESTINO'] = e_combined.apply(reemplazar_ciudad, axis=1)

# Mostrar el DataFrame con las ciudades reemplazadas
e_combined.head()


,VIAC_EMIGRA_INMIGRA,VIAN_NACIONALIDAD,VIAC_SEXO,VIAN_MOTIVO_VIAJE,VIAF_VIAJE,VIAN_PAIS_DESTINO,VIAN_DEPTO_DESTINO,VIAN_CIUDAD_DESTINO,VIAC_ESTADO,VIAC_MEDIO_TRANSPORTE,VIAC_OCUPACION,RANGO_EDAD,REGION_DESTINO,REGION_NACIONALIDAD
0,E,13,M,200,5/5/2018 12:55:13,249,2,MIAMI,V,1,111,3,América del Norte,Asia
1,E,13,F,200,3/19/2018 5:46:55,580,1,CIUDAD DE PANAMA,V,1,1844,2,América Central y el Caribe,Asia
2,E,13,M,218,6/16/2018 6:08:13,249,1,WASHINGTON,V,1,1844,2,América del Norte,Asia
3,E,13,M,124,8/4/2018 11:28:07,105,1,BRASILIA,V,1,111,2,América del Sur,Asia
4,E,13,M,218,6/16/2018 6:06:41,249,1,WASHINGTON,V,1,1844,3,América del Norte,Asia


In [8]:
departamentos_dict = {}

for index, row in departamentos.iterrows():
    pais = row['DEPN_PAISA']
    departamento = row['DEPN_CODIGO']
    depc_nombre = row['DEPC_NOMBRE']
    
    # Si el país no está en el diccionario, agregarlo
    if pais not in departamentos_dict:
        departamentos_dict[pais] = {}
    
    # Asignar el nombre del departamento al código de departamento
    departamentos_dict[pais][departamento] = depc_nombre

In [9]:
# Función para reemplazar los códigos de departamento por los nombres
def reemplazar_departamento(row):
    pais = row['VIAN_PAIS_DESTINO']
    departamento_codigo = row['VIAN_DEPTO_DESTINO']
    
    # Verificar si el país y el departamento están en el diccionario
    if pais in departamentos_dict:
        # Obtener el nombre del departamento si existe, de lo contrario 'Departamento desconocido'
        return departamentos_dict[pais].get(departamento_codigo, 'Departamento desconocido')
    
    return 'Departamento desconocido'

# Reemplazar los códigos de departamento por nombres directamente en el DataFrame
e_combined['VIAN_DEPTO_DESTINO'] = e_combined.apply(reemplazar_departamento, axis=1)

In [10]:
paises_dict = pd.Series(paises['PAIC_NOMBRE'].values, index=paises['PAIN_CODIGO']).to_dict()

In [11]:
e_combined['VIAN_NACIONALIDAD'] = e_combined['VIAN_NACIONALIDAD'].map(paises_dict)
e_combined['VIAN_PAIS_DESTINO'] = e_combined['VIAN_PAIS_DESTINO'].map(paises_dict)

In [12]:
# Asegúrate de que `VIAF_VIAJE` esté en formato de fecha
e_combined['VIAF_VIAJE'] = pd.to_datetime(e_combined['VIAF_VIAJE'], format='%m/%d/%Y %H:%M:%S')

# Crear nuevas columnas para el año y el mes
e_combined['Año'] = e_combined['VIAF_VIAJE'].dt.year
e_combined['Mes'] = e_combined['VIAF_VIAJE'].dt.month
e_combined['Dia'] = e_combined['VIAF_VIAJE'].dt.day

In [13]:
# Asegúrate de que `VIAF_VIAJE` esté en formato de fecha
e_combined['VIAF_VIAJE'] = pd.to_datetime(e_combined['VIAF_VIAJE'], format='%m/%d/%Y %H:%M:%S')

# Contar el número de personas por día
personas_por_dia = e_combined.groupby(e_combined['VIAF_VIAJE'].dt.date).size()

# Calcular el promedio de personas por día
promedio_por_dia = personas_por_dia.mean()

# Contar el número de personas por año
personas_por_año = e_combined.groupby(e_combined['VIAF_VIAJE'].dt.year).size()

# Calcular el promedio de personas por año
promedio_por_año = personas_por_año.mean()

e_combined['VIAF_VIAJE'] = pd.to_datetime(e_combined['VIAF_VIAJE'], format='%m/%d/%Y %H:%M:%S')

# Agrega columnas para año y día
e_combined['Año'] = e_combined['VIAF_VIAJE'].dt.year
e_combined['Fecha'] = e_combined['VIAF_VIAJE'].dt.date

# Cuenta el número de personas por día en cada año
personas_por_dia_anual = e_combined.groupby(['Año', 'Fecha']).size()

# Calcula el promedio de personas por día en cada año
promedio_por_dia_anual = personas_por_dia_anual.groupby(level=0).mean()

print("Promedio de personas por día en cada año:", promedio_por_dia_anual)
print("Promedio de personas por día:", promedio_por_dia)
print("Promedio de personas por año:", promedio_por_año)


Promedio de personas por día en cada año: Año
2018    23194.216438
2019    23138.265753
2020     6339.636612
2021    12219.010959
dtype: float64
Promedio de personas por día: 16216.017796030117
Promedio de personas por año: 5922900.5


In [14]:
e_combined['Año'].value_counts().sort_index()

2018    8465889
2019    8445467
2020    2320307
2021    4459939
Name: Año, dtype: int64

In [15]:
ocupaciones_dict = pd.Series(ocupacion['OCUC_NOMBRE'].values, index=ocupacion['OCUC_CODIGO']).to_dict()
motivo_dict = pd.Series(motivo['MOVC_NOMBRE'].values, index=motivo['MOVN_CODIGO']).to_dict()
medio_transporte_dict = pd.Series(transporte['MEDIO_TRANSPORTE'].values, index=transporte['PUCN_CODIGO']).to_dict()

In [26]:
e_combined['VIAC_MEDIO_TRANSPORTE'] = e_combined['VIAC_MEDIO_TRANSPORTE'].map(medio_transporte_dict)
e_combined['VIAN_MOTIVO_VIAJE'] = e_combined['VIAN_MOTIVO_VIAJE'].map(motivo_dict)
e_combined['VIAC_OCUPACION'] = e_combined['VIAC_OCUPACION'].map(ocupaciones_dict)

In [29]:
def asignar_rango_por_codigo(codigo):
    if codigo == 1:
        return '0 a 17 años'
    elif codigo == 2:
        return '18 a 29 años'
    elif codigo == 3:
        return '30 a 39 años'
    elif codigo == 4:
        return '40 a 49 años'
    elif codigo == 5:
        return '50 a 59 años'
    elif codigo == 6:
        return '60 a 69 años'
    elif codigo == 7:
        return '70 o más'
    else:
        return 'Código desconocido'

# Aplicar la función para reemplazar los códigos en la columna `RANGO_EDAD` por los rangos de edad
e_combined['RANGO_EDAD'] = e_combined['RANGO_EDAD'].apply(asignar_rango_por_codigo)

In [30]:
bogota = e_combined[e_combined['VIAN_CIUDAD_DESTINO'] == 'BOGOTÁ, D.C.']

In [31]:
len(bogota)


29856

In [32]:
bogota.VIAN_NACIONALIDAD.value_counts().head(10)

COLOMBIA           24611
CEUTA Y MELILLA     1107
VENEZUELA           1012
BRASIL               492
ESTADOS UNIDOS       317
ARGENTINA            317
PERU                 312
MEXICO               250
ECUADOR              173
ITALIA               156
Name: VIAN_NACIONALIDAD, dtype: int64

In [33]:
# Asegúrate de que `VIAF_VIAJE` esté en formato de fecha
bogota['VIAF_VIAJE'] = pd.to_datetime(bogota['VIAF_VIAJE'], format='%m/%d/%Y %H:%M:%S')

# Contar el número de personas por día
personas_por_dia = bogota.groupby(bogota['VIAF_VIAJE'].dt.date).size()

# Calcular el promedio de personas por día
promedio_por_dia = personas_por_dia.mean()

# Contar el número de personas por año
personas_por_año = bogota.groupby(bogota['VIAF_VIAJE'].dt.year).size()

# Calcular el promedio de personas por año
promedio_por_año = personas_por_año.mean()

bogota['VIAF_VIAJE'] = pd.to_datetime(bogota['VIAF_VIAJE'], format='%m/%d/%Y %H:%M:%S')

# Agrega columnas para año y día
bogota['Año'] = bogota['VIAF_VIAJE'].dt.year
bogota['Fecha'] = bogota['VIAF_VIAJE'].dt.date

# Cuenta el número de personas por día en cada año
personas_por_dia_anual = bogota.groupby(['Año', 'Fecha']).size()

# Calcula el promedio de personas por día en cada año
promedio_por_dia_anual = personas_por_dia_anual.groupby(level=0).mean()

print("Promedio de personas por día en cada año:", promedio_por_dia_anual)
print("Promedio de personas por día:", promedio_por_dia)
print("Promedio de personas por año:", promedio_por_año)

Promedio de personas por día en cada año: Año
2018    36.583562
2019    41.235616
2020     2.846939
2021     4.265455
dtype: float64
Promedio de personas por día: 27.067996373526746
Promedio de personas por año: 7464.0


C:\Users\Andrés Dardex\AppData\Local\Temp\ipykernel_15336\1798511087.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bogota['VIAF_VIAJE'] = pd.to_datetime(bogota['VIAF_VIAJE'], format='%m/%d/%Y %H:%M:%S')
C:\Users\Andrés Dardex\AppData\Local\Temp\ipykernel_15336\1798511087.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bogota['VIAF_VIAJE'] = pd.to_datetime(bogota['VIAF_VIAJE'], format='%m/%d/%Y %H:%M:%S')
C:\Users\Andrés Dardex\AppData\Local\Temp\ipykernel_15336\1798511087.py:19: SettingWithCopyWa

In [34]:
bogota.head()

,VIAC_EMIGRA_INMIGRA,VIAN_NACIONALIDAD,VIAC_SEXO,VIAN_MOTIVO_VIAJE,VIAF_VIAJE,VIAN_PAIS_DESTINO,VIAN_DEPTO_DESTINO,VIAN_CIUDAD_DESTINO,VIAC_ESTADO,VIAC_MEDIO_TRANSPORTE,VIAC_OCUPACION,RANGO_EDAD,REGION_DESTINO,REGION_NACIONALIDAD,Año,Mes,Dia,Fecha
1069,E,ALEMANIA,M,TURISMO,2018-12-09 14:08:46,COLOMBIA,"BOGOTÁ, D.C.","BOGOTÁ, D.C.",V,AEREO,NaN,50 a 59 años,América del Sur,Europa,2018,12,9,2018-12-09
4169,E,ALEMANIA,M,TURISMO,2018-05-15 06:46:17,COLOMBIA,"BOGOTÁ, D.C.","BOGOTÁ, D.C.",V,AEREO,NaN,40 a 49 años,América del Sur,Europa,2018,5,15,2018-05-15
4321,E,ALEMANIA,F,TURISMO,2018-05-14 11:56:22,COLOMBIA,"BOGOTÁ, D.C.","BOGOTÁ, D.C.",V,AEREO,NaN,60 a 69 años,América del Sur,Europa,2018,5,14,2018-05-14
7251,E,ALEMANIA,M,TURISMO,2018-01-03 13:03:30,COLOMBIA,"BOGOTÁ, D.C.","BOGOTÁ, D.C.",V,AEREO,NaN,50 a 59 años,América del Sur,Europa,2018,1,3,2018-01-03
10824,E,ALEMANIA,M,TURISMO,2018-12-19 11:48:44,COLOMBIA,"BOGOTÁ, D.C.","BOGOTÁ, D.C.",V,AEREO,NaN,40 a 49 años,América del Sur,Europa,2018,12,19,2018-12-19


In [35]:
bogota.to_csv('emigrantes bogota.csv', index=False)